<a href="https://colab.research.google.com/github/samisihem/Projet-de-session/blob/main/Classification%20virale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install biopython

     |████████████████████████████████| 2.3MB 8.1MB/s 


In [2]:
# Import Biopython functions
from Bio import SeqIO

In [3]:
# Initialize an empty dictionary
targets = {}
# Iterate through the entire fasta file
for sequence_record in SeqIO.parse("HIV.fa", "fasta"):
    # Get the index of the separator
    index = sequence_record.id.index(".")
    # Get the target
    target = sequence_record.id[index + 1:]
    # If the target exists, increment its value by 1. 
    if target in targets.keys(): targets[target] = targets[target] + 1
    # Else we add the target to the dictionary with an initial value of 1 
    else: targets[target] = 1

In [4]:
# Get data information before cleaning
n_targets = len(targets.keys())
n_sequences = sum(targets.values())
min_instances = min(targets.values())
max_instances = max(targets.values())

# Display data information before cleaning
print("Data information before cleaning:")
print("Number of sequences = ", n_sequences)
print("Number of targets = ", n_targets)
print("Minimum number of instances = ", min_instances)
print("Maximum number of instances = ", max_instances)

Data information before cleaning:
Number of sequences =  13540
Number of targets =  244
Minimum number of instances =  1
Maximum number of instances =  6792


In [ ]:
#####################################################################################################################
# When we use a cleaning data the accuracy of the classification  is close to 100% from K = 4, for bouth RFE and AE,#
#and it's not easy to do the comparison. For this reason, we keep the data as it is                                 #
#####################################################################################################################

In [ ]:
# Define tables for the cleaning step
#removed_targets = []
#specific_retained_targets = ["A1", "A2", "A3", "A4" "A6", "F1", "F2"]
#specific_excluded_targets = ["-", "U", "A1B", "A1C", "A1CD", "A1D", "02A", "02A1", "BC", "BG", "01B", "01BC", "02B", "BF1", "CD", "01C", "ACD", "02G", "0107", "AD"]


# Iterate through the targets dictionary
#for key, value in targets.items():
    # If the number of instances is greater than ten and the target is not in specific_excluded_targets or if the target is in specific_retained_targets, then pass
 #   if (value >= 10 and key not in specific_excluded_targets) or (key in specific_retained_targets): pass
    # Else add in removed_targets
 #   else: removed_targets.append(key)

# Remove unwanted targets for the dictionary
#for target in removed_targets: del targets[target]

In [5]:
# Initialize our data table [Id, Sequence, Target]
data = []
# Iterate through the entire fasta file
for sequence_record in SeqIO.parse("HIV.fa", "fasta"):
    # Get the index of the separator
    index = sequence_record.id.index(".")
    # Get the target
    target = sequence_record.id[index + 1:]
    # If the current target is in the dictionary
    if target in targets.keys():
        # Initialize a temporary target
        temporary_target = None
        # If subtype A under then merge on the same target
        if target[0] == "A": temporary_target = "A"
        # If subtype F under then merge on the same target
        elif target[0] == "F": temporary_target = "F"
        # Else keep the original target
        else: temporary_target = target
        # Save the information in the data table
        data.append([temporary_target, str(sequence_record.seq).upper(), temporary_target])

In [ ]:
# Updating the dictionary by first clearing it up
#targets.clear()
# Iterate through the entire fasta file
#for d in data:
    # Get the target
 #   target = d[2]
    # If the target exists, increment its value by 1 
  #  if target in targets.keys(): targets[target] = targets[target] + 1
    # Else we add the target to the dictionary with an initial value of 1 
   # else: targets[target] = 1

In [6]:
# Get Data information after selection
n_targets = len(targets.keys())
n_sequences = sum(targets.values())
min_instances = min(targets.values())
max_instances = max(targets.values())

# Dipslay data information after selection
print("Data information after cleaning:")
print("Number of sequences = ", n_sequences)
print("Number of targets = ", n_targets)
print("Minimum number of instances = ", min_instances)
print("Maximum number of instances = ", max_instances)
print("\nData summary:")
for key, value in targets.items(): print("Target = ", key, "| Number of sequences = ", value)

Data information after cleaning:
Number of sequences =  13540
Number of targets =  244
Minimum number of instances =  1
Maximum number of instances =  6792

Data summary:
Target =  B | Number of sequences =  6792
Target =  C | Number of sequences =  2081
Target =  01_AE | Number of sequences =  1420
Target =  02_AG | Number of sequences =  169
Target =  02A1 | Number of sequences =  29
Target =  01B | Number of sequences =  216
Target =  01BC | Number of sequences =  51
Target =  BC | Number of sequences =  94
Target =  A1 | Number of sequences =  497
Target =  G | Number of sequences =  92
Target =  02A3 | Number of sequences =  7
Target =  02D | Number of sequences =  6
Target =  BF1 | Number of sequences =  142
Target =  CU | Number of sequences =  5
Target =  06_cpx | Number of sequences =  14
Target =  0206 | Number of sequences =  9
Target =  02G | Number of sequences =  17
Target =  D | Number of sequences =  86
Target =  F1 | Number of sequences =  76
Target =  U | Number of se

In [ ]:
##################################################
##### GENERATION OF FEATURES BASED ON K-MERS #####
##################################################


In [7]:
# Import re for regular expression operations
import re
# Fixe the length k of the features based on k-mers
k = 5 #3,4,5
# Initialize an empty dictionary for the k-mers
k_mers = {}

In [8]:
# Iterate through the training data
for d in data:
    # Get the sequence
    sequence = d[1]
    # Go through the sequence 
    for i in range(0, len(sequence) - k + 1, 1):
        # Get the current k-mer
        k_mer = sequence[i:i + k]
        # If it contains only the characters "A", "C", "G" or "T", it will be saved.
        if bool(re.match('^[ACGT]+$', k_mer)) == True: k_mers[k_mer] = 0

In [9]:
def generateMatrices(data, k_mers, k):
    # Initialize the feature matrix
    X = []
    # Initialize the target vector
    y = []
    # Iterate through the data
    for d in data:
        # Generate an empty dictionary
        x = {}
        # Initialize the dictionary with targets as keys and 0 as value
        x = x.fromkeys(k_mers.keys(), 0)
        # Get the sequence
        sequence = d[1]
        # Get the target
        target = d[2]
        # Compute X (features matrix) the number of occurrence of k-mers (with overlaping)
        for i in range(0, len(sequence) - k + 1, 1):
            k_mer = sequence[i:i + k]
            # Attempt to increment the number of occurrences of the current k-mers
            try: x[k_mer] = x[k_mer] + 1
            except: pass
        # Save the vector in the main matrix
        X.append(list(x.values()))
        # Compute y (target vector)
        y.append(target)
    # Return matrices X and y (feature matrix and target vector)
    return X, y

In [10]:
seq_X, seq_y = generateMatrices(data = data, k_mers = k_mers, k = k)

In [11]:
# Import MinMaxScaler from sklearn
from sklearn.preprocessing import MinMaxScaler
# Instantiate a MinMaxScaler between 0 and 1
minMaxScaler = MinMaxScaler(feature_range = (0,1))
# Apply a scaling to the train and test set
seq_X = minMaxScaler.fit_transform(seq_X)

In [ ]:
###################################################################################################################
#Since we want to do a comparison, and in order to have a fast execution in the selection of attributes with RFE, #
#we choose 30% for trainning and 70% for the test. The same trainning and test data are using for bouth RFE and AE# 
###################################################################################################################

In [24]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
rs = 42
test_size = 0.7

X_train, X_test, y_train, y_test = train_test_split(seq_X, seq_y, 
                                                     test_size=test_size, 
                                                     shuffle=True, 
                                                     random_state=rs)
print(X_train.shape)

(4062, 1024)


In [14]:
# Import SVM from sklearn
from sklearn import svm
# Instantiate a linear model based on svm
model = svm.SVC(C = 1.0, kernel='linear', class_weight = None)

In [ ]:
# Imports the feature selection method: Recursive features elimination [RFE]
import time
start = time.clock()
print(start)

from sklearn.feature_selection import RFE
# Define the elimination step
step = 2
# Define the number of features to be selected
n_features = 100 # 32, 60, 100
# Instantiate the RFE
rfe = RFE(model, n_features_to_select = n_features, step = step)
# Apply RFE and transform the training matrix
X_train = rfe.fit_transform(X_train, y_train)

In [16]:
# Fit the model on the train set
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:
# Import Joblib for the persistence of the model
import joblib

In [19]:
# Save the model
joblib.dump(model, "model.pkl")

['model.pkl']

In [20]:
# Transform the test matrix according to the RFE selection
X_test = rfe.transform(X_test)

In [21]:
# Load model
model = joblib.load("model.pkl")

In [22]:
# Predict the test set
y_pred = model.predict(X_test)

In [23]:
# Import classification report from sklearn
from sklearn.metrics import classification_report
# Print the classification_report
print(classification_report(y_test, y_pred, digits = 3))
# Save the keys of the classification report dictionary 
classification_report_dict_keys = list(classification_report(y_test, y_pred, output_dict= True).keys())[:-3]

              precision    recall  f1-score   support

           -      0.820     0.631     0.713        65
        0102      0.000     0.000     0.000         3
       0102A      0.000     0.000     0.000         3
      0102A1      0.000     0.000     0.000         1
        0107      0.595     0.543     0.568        46
        0108      0.750     0.750     0.750         4
      01A1A6      0.000     0.000     0.000         1
      01A1F2      0.000     0.000     0.000         1
      01ADF2      0.000     0.000     0.000         1
         01B      0.627     0.723     0.671       137
        01BC      0.714     0.395     0.508        38
         01C      0.667     0.222     0.333         9
       01F1G      1.000     1.000     1.000         1
        01F2      0.000     0.000     0.000         1
       01F2G      0.000     0.000     0.000         1
       01_AE      0.955     0.987     0.971      1005
        0206      0.750     0.500     0.600         6
        0209      0.000    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle
from keras.optimizers import SGD

In [ ]:
###############################
# Different network for each k #
###############################

In [ ]:
# K= 3, with 64 total features
# select 32 features
encoding_dim = 32 # 32, 60, 100   
# this is our input placeholder
input_data = Input(shape=(64, ))
# Number of epochs
my_epochs = 200

# "encoded" is the encoded representation of the inputs
encoded = Dense(44, activation='relu')(input_data)
encoded = Dense(encoding_dim , activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(44, activation='relu')(encoded)
decoded = Dense(64, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_data, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_data, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))

# retrieve the layers of the autoencoder model
decoder_layer3 = autoencoder.layers[-2]
decoder_layer4 = autoencoder.layers[-1]

# create the decoder model
decoder = Model(encoded_input, decoder_layer4(decoder_layer3(encoded_input)))

# configure model to use regretion loss functions, and the adam optimizer
autoencoder.compile(optimizer='adam', loss='mse')
#mean_squared_logarithmic_error, #mean_absolute_error, #autoencoder.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['mse'])

In [ ]:
# K= 4, with 256 total features
# select 60 features
encoding_dim = 60 # 32, 60, 100   
# this is our input placeholder
input_data = Input(shape=(256, ))
# Number of epochs
my_epochs = 300

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 3, activation='relu')(input_data)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim , activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 3, activation='relu')(decoded)
decoded = Dense(256, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_data, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_data, encoded)

# create a placeholder for an encoded (60-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))

# retrieve the layers of the autoencoder model
decoder_layer2 = autoencoder.layers[-3]
decoder_layer3 = autoencoder.layers[-2]
decoder_layer4 = autoencoder.layers[-1]

# create the decoder model
decoder = Model(encoded_input, decoder_layer4(decoder_layer3(decoder_layer2(encoded_input))))

# configure model to use regretion loss functions, and the adam optimizer
autoencoder.compile(optimizer='adam', loss='mse')
#autoencoder.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['mse'])
#mean_squared_logarithmic_error, #mean_absolute_error,

In [65]:
# K= 5, with 1024 total features
# select 100 features
encoding_dim = 100 # 32, 60, 100   
# this is our input placeholder
input_data = Input(shape=(1024, ))
# Number of 300
my_epochs = 300

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim *8, activation='relu')(input_data)
encoded = Dense(encoding_dim *5 , activation='relu')(encoded)
encoded = Dense(encoding_dim *3 , activation='relu')(encoded)
encoded = Dense(encoding_dim *2 , activation='relu')(encoded)
encoded = Dense(encoding_dim , activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim*2, activation='relu')(encoded)
decoded = Dense(encoding_dim*3, activation='relu')(decoded)
decoded = Dense(encoding_dim*5, activation='relu')(decoded)
decoded = Dense(encoding_dim*8, activation='relu')(decoded)
decoded = Dense(1024, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_data, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_data, encoded)

# create a placeholder for an encoded (100-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))

# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-5]
decoder_layer2 = autoencoder.layers[-4]
decoder_layer3 = autoencoder.layers[-3]
decoder_layer4 = autoencoder.layers[-2]
decoder_layer5 = autoencoder.layers[-1]


# create the decoder model
decoder = Model(encoded_input, decoder_layer5(decoder_layer4(decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))))

# configure model to use regretion loss functions, and the adam optimizer

autoencoder.compile(optimizer= 'adam', loss='mse')
K.set_value(autoencoder.optimizer.learning_rate, 0.0003)
#autoencoder.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['mse'])
#mean_squared_logarithmic_error, #mean_absolute_error,

In [66]:

# Train autoencoder
autoencoder.fit(seq_X, seq_X, epochs=my_epochs, batch_size=256, shuffle=True, validation_data=None,verbose=2)

K.clear_session()

Epoch 1/300
53/53 - 4s - loss: 0.0254
Epoch 2/300
53/53 - 3s - loss: 0.0163
Epoch 3/300
53/53 - 3s - loss: 0.0159
Epoch 4/300
53/53 - 3s - loss: 0.0143
Epoch 5/300
53/53 - 3s - loss: 0.0132
Epoch 6/300
53/53 - 3s - loss: 0.0124
Epoch 7/300
53/53 - 3s - loss: 0.0122
Epoch 8/300
53/53 - 3s - loss: 0.0121
Epoch 9/300
53/53 - 3s - loss: 0.0119
Epoch 10/300
53/53 - 3s - loss: 0.0117
Epoch 11/300
53/53 - 3s - loss: 0.0115
Epoch 12/300
53/53 - 3s - loss: 0.0112
Epoch 13/300
53/53 - 3s - loss: 0.0110
Epoch 14/300
53/53 - 3s - loss: 0.0108
Epoch 15/300
53/53 - 3s - loss: 0.0106
Epoch 16/300
53/53 - 3s - loss: 0.0105
Epoch 17/300
53/53 - 3s - loss: 0.0103
Epoch 18/300
53/53 - 3s - loss: 0.0101
Epoch 19/300
53/53 - 3s - loss: 0.0099
Epoch 20/300
53/53 - 3s - loss: 0.0098
Epoch 21/300
53/53 - 3s - loss: 0.0096
Epoch 22/300
53/53 - 3s - loss: 0.0095
Epoch 23/300
53/53 - 3s - loss: 0.0094
Epoch 24/300
53/53 - 3s - loss: 0.0092
Epoch 25/300
53/53 - 3s - loss: 0.0091
Epoch 26/300
53/53 - 3s - loss: 0.

In [67]:
# get the encoded representation
encoded_imgs = encoder.predict(X_train)
Data_train = encoded_imgs[:]

In [68]:
Data_train.shape

(4062, 100)

In [69]:
# Import SVM from sklearn
from sklearn import svm
# Instantiate a linear model based on svm
model2 = svm.SVC(C = 1.0, kernel='linear', class_weight = None)

In [70]:
model2.fit(Data_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [71]:
# Import Joblib for the persistence of the model
import joblib

In [72]:
# Save the model
joblib.dump(model2, "model2.pkl")

['model2.pkl']

In [73]:
# Load model
model2 = joblib.load("model2.pkl")

In [74]:
# Predict the test set
encoded_imgs = encoder.predict(X_test)
Data_test = encoded_imgs[:]
y_pred = model2.predict(Data_test)

In [75]:
Data_test.shape

(9478, 100)

In [76]:
# Import classification report from sklearn
from sklearn.metrics import classification_report
# Print the classification_report
print(classification_report(y_test, y_pred, digits = 3))
# Save the keys of the classification report dictionary 
classification_report_dict_keys = list(classification_report(y_test, y_pred, output_dict= True).keys())[:-3]

              precision    recall  f1-score   support

           -      0.909     0.769     0.833        65
        0102      0.000     0.000     0.000         3
       0102A      0.000     0.000     0.000         3
      0102A1      0.000     0.000     0.000         1
        0107      0.674     0.674     0.674        46
        0108      0.750     0.750     0.750         4
       0122F      0.000     0.000     0.000         0
      01A1A6      0.000     0.000     0.000         1
      01A1F2      0.000     0.000     0.000         1
       01A1G      0.000     0.000     0.000         0
      01ADF2      0.000     0.000     0.000         1
         01B      0.669     0.854     0.750       137
        01BC      0.531     0.447     0.486        38
         01C      0.333     0.333     0.333         9
         01D      0.000     0.000     0.000         0
       01F1G      1.000     1.000     1.000         1
        01F2      0.000     0.000     0.000         1
       01F2G      0.000    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
